In [6]:
%matplotlib inline

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
from torch.utils.data import Dataset

class VNdata(Dataset):
    def __init__(self, pt_file):
        self.datafile = torch.load(pt_file)
        self.input = []
        for key in self.datafile.keys():
            item = self.datafile[key][0]
            self.input.append((item['verb'], item['noun'], item['action_label']))

    def __getitem__(self, index):
        verb_score, noun_score, action_label = self.input[index]
        return verb_score, noun_score, action_label

    def __len__(self):
        return len(self.input)

In [41]:
from torch.utils.data import DataLoader
train_pt = '/data/usrdata/roy/checkpoints/hoivit_ek_wmotion_doubleatt_ant/EPIC_100_train.pt'
val_pt = '/data/usrdata/roy/checkpoints/hoivit_ek_wmotion_doubleatt_ant/EPIC_100_validation.pt'
train_set = VNdata(train_pt)
val_set = VNdata(val_pt)

trainloader = DataLoader(val_set, batch_size=64, shuffle=True)
valloader = DataLoader(train_set, batch_size=64, shuffle=False)

In [42]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(97, 512)  
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(300, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(1024, 3806)
        self.dropout = nn.Dropout()
    def forward(self, verb, noun):
        verb = F.relu(self.bn1(self.fc1(verb)))
        noun = F.relu(self.bn2(self.fc2(noun)))
        out = torch.concat((verb,noun),dim=1)
        # print(out.shape)
        out = self.dropout(out)
        out = self.fc3(out)
        return out


net = Net()
device = torch.cuda.current_device()
net.to(device)

Net(
  (fc1): Linear(in_features=97, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=300, out_features=512, bias=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=1024, out_features=3806, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [43]:
import torch.optim as optim
import numpy as np

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(net.parameters(), lr=0.01)
num_classes = 3806

loss_list = []
for epoch in range(100):
    class_wise_frequency = np.zeros(num_classes)
    class_wise_correct5 = np.zeros(num_classes)
    for i, data in enumerate(trainloader):
        verb_score, noun_score, action_label = data
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(verb_score.to(device), noun_score.to(device))
        loss = criterion(output, action_label.to(device))
        loss_list.append(loss.item())
        loss.backward()
        optimizer.step()    # Does the update

        output = output.detach().cpu()    
        _, pred5 = output.topk(k=5, dim=-1)
        
        target5 = action_label.unsqueeze(dim=-1).expand_as(pred5)
        correct5 = target5[pred5 == target5] # [] if different and class_num if same
        
        for cls in range(num_classes):
            class_wise_frequency[cls] += (action_label == cls).sum().item()
            class_wise_correct5[cls] += (correct5 == cls).sum().item()
    
    class_wise_recall5 = []
    for cls in range(num_classes):
        if class_wise_frequency[cls] != 0:
           class_wise_recall5.append(class_wise_correct5[cls] / class_wise_frequency[cls])

    mean_recall5 = torch.Tensor(class_wise_recall5).mean()*100

    print('[Train Epoch {} [Action Mean Recall @5: {:.2f}]'.format(epoch, mean_recall5.item()))

    class_wise_frequency = np.zeros(num_classes)
    class_wise_correct5 = np.zeros(num_classes)
    with torch.no_grad():
        for i, data in enumerate(valloader):
            verb_score, noun_score, action_label = data
            output = net(verb_score.to(device), noun_score.to(device))
            
            output = output.detach().cpu()    
            _, pred5 = output.topk(k=5, dim=-1)
            
            target5 = action_label.unsqueeze(dim=-1).expand_as(pred5)
            correct5 = target5[pred5 == target5] # [] if different and class_num if same
            
            for cls in range(num_classes):
                class_wise_frequency[cls] += (action_label == cls).sum().item()
                class_wise_correct5[cls] += (correct5 == cls).sum().item()
    
    class_wise_recall5 = []
    for cls in range(num_classes):
        if class_wise_frequency[cls] != 0:
           class_wise_recall5.append(class_wise_correct5[cls] / class_wise_frequency[cls])

    mean_recall5 = torch.Tensor(class_wise_recall5).mean()*100

    print('[Val Epoch {} [Action Mean Recall @5: {:.2f}]'.format(epoch, mean_recall5.item()))

[Train Epoch 0 [Action Mean Recall @5: 2.95]
[Train Epoch 1 [Action Mean Recall @5: 5.79]
[Train Epoch 2 [Action Mean Recall @5: 8.34]
[Train Epoch 3 [Action Mean Recall @5: 11.09]
[Train Epoch 4 [Action Mean Recall @5: 15.38]
[Train Epoch 5 [Action Mean Recall @5: 20.67]
[Train Epoch 6 [Action Mean Recall @5: 28.84]
[Train Epoch 7 [Action Mean Recall @5: 36.37]
[Train Epoch 8 [Action Mean Recall @5: 43.84]
[Train Epoch 9 [Action Mean Recall @5: 51.58]
[Train Epoch 10 [Action Mean Recall @5: 56.61]


KeyboardInterrupt: 

In [1]:
import matplotlib.pyplot as plt
loss_list = np.array(loss_list)
step = np.linspace(0, loss_list.shape[0], loss_list.shape[0])
plt.plot(step,loss_list)

NameError: name 'np' is not defined